# Can we improve model accuracy by using a smaller model?

Using fastai `XResNet` we built a model smaller than `ResNeXt 18 deep` that improved accuracy by ~0.5% after 5 epochs of Imagenette training.

| model              | seconds per epoch | accuracy |
| -------------------|-------------------|----------|
| xse_resnext50      | 62                | 84.8%    |
| xse_resnext18_deep | 20                | 84.8%    |
| mini net           | 17                | 85.3%    |

Note: this notebook is based on fastai2/nbs/examples/train_imagenette.py

In [1]:
import PIL # hack to re-instate PILLOW_VERSION (thanks pillow 7 :o)
PIL.PILLOW_VERSION = PIL.__version__

from fastai2.basics import *
from fastai2.vision.all import *
from fastai2.callback.all import *
from fastai2.distributed import *
from fastprogress import fastprogress
from torchvision.models import *
from fastai2.vision.models.xresnet import *
from fastai2.callback.mixup import *
from fastscript import *

torch.backends.cudnn.benchmark = True
fastprogress.MAX_COLS = 80

In [2]:
def get_dls(size, woof, bs, sh=0., workers=None):
    if size<=224: path = URLs.IMAGEWOOF_320 if woof else URLs.IMAGENETTE_320
    else        : path = URLs.IMAGEWOOF     if woof else URLs.IMAGENETTE
    source = untar_data(path)
    if workers is None: workers = min(8, num_cpus())
    # Resize seems to give slightly better accuracy than RandomResizedCrop
    resize_ftm = Resize(size) # RandomResizedCrop(size, min_scale=0.35)
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                       splitter=GrandparentSplitter(valid_name='val'),
                       get_items=get_image_files, get_y=parent_label,
                       item_tfms=[resize_ftm, FlipItem(0.5)],
                       batch_tfms=RandomErasing(p=0.3, max_count=3, sh=sh) if sh else None)
    return dblock.dataloaders(source, path=source, bs=bs, num_workers=workers)

In [3]:
@call_parse
def main(
    gpu:   Param("GPU to run on", int)=None,
    woof:  Param("Use imagewoof (otherwise imagenette)", int)=0,
    lr:    Param("Learning rate", float)=1e-2,
    size:  Param("Size (px: 128,192,256)", int)=128,
    sqrmom:Param("sqr_mom", float)=0.99,
    mom:   Param("Momentum", float)=0.9,
    eps:   Param("epsilon", float)=1e-6,
    epochs:Param("Number of epochs", int)=5,
    bs:    Param("Batch size", int)=64,
    mixup: Param("Mixup", float)=0.,
    opt:   Param("Optimizer (adam,rms,sgd,ranger)", str)='ranger',
    arch:  Param("Architecture", str)='xresnet50',
    sh:    Param("Random erase max proportion", float)=0.,
    sa:    Param("Self-attention", int)=0,
    sym:   Param("Symmetry for self-attention", int)=0,
    beta:  Param("SAdam softplus beta", float)=0.,
    act_fn:Param("Activation function", str)='MishJit',
    fp16:  Param("Use mixed precision training", int)=0,
    pool:  Param("Pooling method", str)='AvgPool',
    dump:  Param("Print model; don't train", int)=0,
    runs:  Param("Number of times to repeat training", int)=1,
    meta:  Param("Metadata (ignored)", str)='',
    wd:    Param("Weight decay", float)=1e-2
):
    "Training of Imagenette."

    #gpu = setup_distrib(gpu)
    if gpu is not None: torch.cuda.set_device(gpu)
    if   opt=='adam'  : opt_func = partial(Adam, mom=mom, sqr_mom=sqrmom, eps=eps)
    elif opt=='rms'   : opt_func = partial(RMSprop, sqr_mom=sqrmom)
    elif opt=='sgd'   : opt_func = partial(SGD, mom=mom)
    elif opt=='ranger': opt_func = partial(ranger, mom=mom, sqr_mom=sqrmom, eps=eps, beta=beta)

    dls = get_dls(size, woof, bs, sh=sh)
    if not gpu: 
        print(f'epochs: {epochs}; lr: {lr}; size: {size}; sqrmom: {sqrmom}; mom: {mom}; eps: {eps}')
        print(f'fp16: {fp16}; arch: {arch}; wd: {wd}; act_fn: {act_fn}; bs: {bs}')
        print(f'pool: {pool}; woof: {woof}; sh:{sh}')
        
    m,act_fn,pool = [globals()[o] for o in (arch,act_fn,pool)]

    for run in range(runs):
        print(f'Run: {run}')
        learn = Learner(dls, m(c_out=10, act_cls=act_fn, sa=sa, sym=sym, pool=pool), opt_func=opt_func, \
                metrics=[accuracy,top_k_accuracy], loss_func=LabelSmoothingCrossEntropy())
        if dump: print(learn.model); exit()
        if fp16: learn = learn.to_fp16()
        cbs = MixUp(mixup) if mixup else []
        #n_gpu = torch.cuda.device_count()
        #if gpu is None and n_gpu: learn.to_parallel()
        if num_distrib()>1: learn.to_distributed(gpu) # Requires `-m fastai.launch`
        learn.fit_flat_cos(epochs, lr, wd=wd, cbs=cbs)

## Try a small "out of the box" model to see what we have to beat

`ResNeXt 18 deep` seems to be pretty good to start with. Reducing weight decay improves accuracy a little.

In [4]:
lr=1e-2
arch='xse_resnext18_deep'
wd=1e-4
main(lr=lr, arch=arch, wd=wd)

epochs: 5; lr: 0.01; size: 128; sqrmom: 0.99; mom: 0.9; eps: 1e-06
fp16: 0; arch: xse_resnext18_deep; wd: 0.0001; act_fn: MishJit; bs: 64
pool: AvgPool; woof: 0; sh:0.0
Run: 0


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.476539,1.757093,0.504713,0.912611,00:22
1,1.236627,1.185492,0.720255,0.966879,00:20
2,1.112664,1.212075,0.711083,0.953631,00:20
3,1.019457,0.999184,0.800255,0.977070,00:20
4,0.851736,0.890027,0.847898,0.983440,00:21


## Try a smaller model

Reducing layers from `[2,2,2,2,1,1]` to `[1,1,1,1]` improved accuracy and made training faster ... but I had to fiddle with `groups` and `reduction` to get consitently better results.

In [13]:
def mini_net(c_out=1000, pretrained=False, **kwargs):
    block=SEResNeXtBlock
    expansion=1
    layers=[1,1,1,1] # [2,2,2,2,1,1] xse_resnext18_deep
    groups=64        # 32 
    reduction=8      # 16
    print(f'block={block} expansion={expansion} layers={layers} groups={groups} reduction={reduction}')
    return XResNet(block, expansion, layers, c_out=c_out, groups=groups, reduction=reduction, **kwargs)

globals()['mini_net'] = mini_net

With layers=[1, 1, 1, 1], groups=32 and reduction=16 we see nearly 0.86 accuracy most of the time - but it sometimes drops to ~0.84.

I think the changes to `groups` and `reduction` improved the consistency of `mini net` - but may have reduced peak accuracy a tiny bit.

Note: I tried a few changes to `groups` and `reduction` with `xse_resnext18_deep` but anything other than 32/16 made it worse.

In [14]:
arch='mini_net'
main(runs=1, lr=lr, arch=arch, wd=wd)

epochs: 5; lr: 0.01; size: 128; sqrmom: 0.99; mom: 0.9; eps: 1e-06
fp16: 0; arch: mini_net; wd: 0.0001; act_fn: MishJit; bs: 64
pool: AvgPool; woof: 0; sh:0.0
Run: 0
block=<function SEResNeXtBlock at 0x7f9e61ab5cb0> expansion=1 layers=[1, 1, 1, 1] groups=64 reduction=8


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.573084,1.666911,0.539108,0.907261,00:17
1,1.266548,1.149882,0.742675,0.969936,00:17
2,1.118694,1.258321,0.696306,0.954904,00:17
3,1.004780,1.002810,0.809172,0.980892,00:17
4,0.852620,0.898029,0.852994,0.987261,00:17


let's try this config over 5 runs ...

In [15]:
arch='mini_net'
main(runs=5, lr=lr, arch=arch, wd=wd)

epochs: 5; lr: 0.01; size: 128; sqrmom: 0.99; mom: 0.9; eps: 1e-06
fp16: 0; arch: mini_net; wd: 0.0001; act_fn: MishJit; bs: 64
pool: AvgPool; woof: 0; sh:0.0
Run: 0
block=<function SEResNeXtBlock at 0x7f9e61ab5cb0> expansion=1 layers=[1, 1, 1, 1] groups=64 reduction=8


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.574513,2.064940,0.441783,0.897580,00:17
1,1.279880,1.160462,0.732484,0.971720,00:17
2,1.091039,1.353692,0.667006,0.950318,00:17
3,0.995791,1.009981,0.807898,0.978854,00:17
4,0.858089,0.890406,0.857325,0.987261,00:17


Run: 1
block=<function SEResNeXtBlock at 0x7f9e61ab5cb0> expansion=1 layers=[1, 1, 1, 1] groups=64 reduction=8


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.553005,1.747727,0.526879,0.908026,00:17
1,1.277722,1.154143,0.735032,0.968153,00:17
2,1.115307,1.401358,0.663185,0.948025,00:17
3,1.015072,0.980753,0.815796,0.981911,00:17
4,0.851516,0.896918,0.849427,0.986497,00:18


Run: 2
block=<function SEResNeXtBlock at 0x7f9e61ab5cb0> expansion=1 layers=[1, 1, 1, 1] groups=64 reduction=8


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.552149,1.736750,0.489936,0.911847,00:18
1,1.281032,1.164669,0.730701,0.967389,00:18
2,1.112261,1.320213,0.655032,0.951338,00:18
3,1.022199,0.990990,0.812484,0.979618,00:17
4,0.868079,0.900094,0.851210,0.986242,00:18


Run: 3
block=<function SEResNeXtBlock at 0x7f9e61ab5cb0> expansion=1 layers=[1, 1, 1, 1] groups=64 reduction=8


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.554691,2.110049,0.398981,0.926115,00:17
1,1.279650,1.185858,0.715924,0.966879,00:17
2,1.093604,1.186145,0.724841,0.964841,00:17
3,1.002387,0.987181,0.814777,0.979108,00:17
4,0.852337,0.896359,0.853758,0.985732,00:17


Run: 4
block=<function SEResNeXtBlock at 0x7f9e61ab5cb0> expansion=1 layers=[1, 1, 1, 1] groups=64 reduction=8


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.597253,1.699681,0.517452,0.896306,00:17
1,1.271300,1.190564,0.730701,0.969682,00:17
2,1.115927,1.215999,0.714395,0.965605,00:18
3,1.012310,1.006783,0.807898,0.978854,00:18
4,0.856653,0.901137,0.851720,0.985987,00:18


## find the mean of the last 5 runs

In [16]:
res = [0.857325, 0.849427, 0.851210, 0.853758, 0.851720]
print(np.mean(res), np.median(res))

0.852688 0.85172
